In [278]:
import pandas as pd
from binance.client import Client
from Binance_api import api_key, secret_key
from datetime import datetime
from datetime import timedelta
import time
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
import plot_utils 
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

warnings.filterwarnings('ignore')

client = Client(api_key, secret_key,tld='us')
rsi = 0

In [279]:
df = pd.read_csv('BTC_5min_10AUG.csv')

In [280]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band

symbols = ['BTC']

nstd = 3
def rsi(symbol, i):
    if df[f'{symbol}_rsi'].iloc[i] > rsi:
        rsi = df[f'{symbol}_rsi'].iloc[i]
    return rsi

        
    



for symbol in symbols:
    df[f'{symbol}_sma'] = sma(df[f'{symbol}-USD_Open'],20)
    df[f'{symbol}_upper_band'], df[f'{symbol}_lower_band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_sma'], 20, nstd)        
    df[f'{symbol}_rsi'] = ta.RSI(df[f'{symbol}-USD_Close'])

        
df.dropna(inplace=True)

In [281]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = trading_fee_multiplier
        
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
        
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append( [self.balance_unit, time, sell_price] )
        self.balance_unit = 'USDT'

# VIP level 0, paying fees with BNB = 0.075%

        

In [282]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=1) #0.99925

In [283]:
last_buy = 0
buy = 0
sell = 0
d = 0
for i in range(len(df)):
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] < 30: #buy signal
                buy = 1
                d = d + 1
            if buy == 1 and df[f'{symbol}-USD_Open'].iloc[i] <  df[f'{symbol}-USD_Close'].iloc[i]:
                env.buy(symbol, df[f'{symbol}-USD_Close'].iloc[i], df['OpenTime'].iloc[i])
                last_buy = df[f'{symbol}_lower_band'].iloc[i]
                buy = 0
                break

    if env.balance_unit != 'USDT':
        if last_buy*0.98 > df[f'{env.balance_unit}-USD_Close'].iloc[i]:
            env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i])
            sell = 0
        elif df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] > 60: #sell signal
            sell = 1
        if sell == 1 and df[f'{symbol}-USD_Open'].iloc[i] > df[f'{symbol}-USD_Close'].iloc[i]:
            env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i])
            sell = 0

if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])

In [284]:
d

285

In [285]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print('starting balance: 100 USDT')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')


num buys: 162
num sells: 162
starting balance: 100 USDT
ending balance: 87.88528877497467 USDT


In [286]:
balance = 100
profit_total = 0
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(buys)
    print(sells)
    print('P/L = 'f'{profit}')
    balance += balance*(profit/100)
    print(balance)
    print('\n')
    

['BTC', '2022-08-11 04:50:00', 23544.07]
['BTC', '2022-08-11 07:00:00', 23825.78]
P/L = 1.196522096646846
101.19652209664685


['BTC', '2022-08-12 04:25:00', 24197.96]
['BTC', '2022-08-12 07:35:00', 24163.07]
P/L = -0.14418570821672638
101.0506111745711


['BTC', '2022-08-12 09:00:00', 23915.68]
['BTC', '2022-08-12 12:55:00', 24011.51]
P/L = 0.40069945742708946
101.45552042527437


['BTC', '2022-08-13 19:05:00', 24396.74]
['BTC', '2022-08-14 01:10:00', 24494.49]
P/L = 0.40066828600870963
101.86202052002353


['BTC', '2022-08-14 21:45:00', 24563.87]
['BTC', '2022-08-15 07:15:00', 24349.59]
P/L = -0.872338112846216
100.97343929251214


['BTC', '2022-08-15 16:40:00', 24377.12]
['BTC', '2022-08-15 20:00:00', 24163.18]
P/L = -0.8776262331235185
100.087269900794


['BTC', '2022-08-16 09:15:00', 23847.26]
['BTC', '2022-08-16 19:25:00', 24083.44]
P/L = 0.99038631691859
101.07852052686884


['BTC', '2022-08-17 00:35:00', 23744.42]
['BTC', '2022-08-17 04:10:00', 23964.37]
P/L = 0.926322900285625

0
1
2
